# Land Change
<img src="https://arcgis01.satapps.org/portal/sharing/rest/content/items/a499849ccd1f4c7fb0403b4c719f9dc1/resources/Land%20Degradation.png" />
[find out more](https://arcgis01.satapps.org/portal/apps/sites/?fromEdit=true#/data/pages/data-cube)

This product uses changes in Fractional Cover to identify land change. The algorithm identifies a "baseline" and "analysis" time period and then compares the fractions in each of those time periods.

Fractional Cover represents the proportion of the land surface which is bare (BS), covered by photosynthetic vegetation (PV), or non-photosynthetic vegetation(NPV). 

The Fractional Cover product was generated using the spectral unmixing algorithm developed by the Joint Remote Sensing Research Program (JRSRP) which used the spectral signature for each pixel to break it up into three fractions, based on field work that determined the spectral characteristics of these fractions. The fractions were retrieved by inverting multiple linear regression estimates and using synthetic endmembers in a constrained non-negative least squares unmixing model.

The green (PV) fraction includes leaves and grass, the non-photosynthetic fraction (NPV) includes branches, dry grass and dead leaf litter, and the bare soil (BS) fraction includes bare soil or rock.

Changes in each fraction are conincident with land change.

In some cases these changes could be deforestation. Users of this algorithm should not accept the accuracy of the results but should conduct ground validation testing to assess accuracy. In most cases, these algorithms can be used to identify clusters of pixels that have experienced change and allow targeted investigation of those areas by local or regional governments.

This output of this notebook is a raster product for each of the fractional cover bands - where positive changes represents gain in that band, and negative change represents loss. 

In [1]:
# jupyteronly

%load_ext autoreload
%autoreload 2
%matplotlib inline
# Supress Warnings 
import warnings
warnings.filterwarnings('ignore')

import datacube
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt

from matplotlib.cm import RdYlGn, Greens
#from datacube_utilities.dc_display_map import display_map

import glob
import yaml
import rioxarray as rxr

In [2]:
# Magic + imports likely common across all notebooks
from odc.algo import to_f32, from_float, xr_geomedian
from datacube.utils.cog import write_cog

from pyproj import Proj, transform

# Generic python
import numpy as np
import xarray as xr 
import odc.algo
import dask
from dask.distributed import Client

# Bonus vector manipulation
from shapely import wkt
from datetime import datetime

# Import functions to load and stack data without datacube
from notebook_functions import *
from utilities import *

client = Client(n_workers=2, threads_per_worker=4, memory_limit='7GB')

client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 2
Total threads: 8,Total memory: 13.04 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:46365,Workers: 2
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 13.04 GiB
Comm: tcp://127.0.0.1:42583,Total threads: 4
Dashboard: http://127.0.0.1:35461/status,Memory: 6.52 GiB
Nanny: tcp://127.0.0.1:34247,


## Set Up Parameters

In [3]:
# Running locally on landsat 8 data for now
product = 'landsat_8'

# St Maarten bounding box to subset the data
clip_coords = {'min_lon':-63.461424,
               'min_lat': 17.950000,
               'max_lon': -62.80000,
               'max_lat': 18.334848}

# Set size of dask chunks to use for the scenes
dask_chunks = dict(
    x = 1000,
    y = 1000
)

# Set mosaic type (options: 'median', 'mean', 'max', 'min')
mosaic_type = 'median'


measurements = ["blue", "green", "red", "nir", "swir1", "swir2", "pixel_qa"]


#parameter display_name="Water Mask" description="If you would like the water to be masked out choose YES, if you would like the full image choose NO" datatype="string" options=["YES", "NO"],
mask_water = 'YES'


## Load Data

In [4]:
def prep_dataset(in_dir, measurement, product, clip_coords = None):
    """Prepare either the baseline or analysis dataset."""
    scenes = glob.glob(f'{in_dir}/*/')

    array_list = []

    for scene in scenes:
        yml = f'{scene}/datacube-metadata.yaml'
        with open (yml) as stream: yml_meta = yaml.safe_load(stream)

        # Load the bands provided in 'measurement' from the yaml file.
        o_bands_data = [ rxr.open_rasterio(scene + yml_meta['image']['bands'][b]['path'], chunks=dask_chunks) for b in measurement ] 

        # Clip the data to the bounding box if provided.
        if clip_coords is not None:
            o_bands_data = [ o_bands_data[i].rio.clip_box(minx = clip_coords['min_lon'], miny = clip_coords['min_lat'], 
                                                          maxx = clip_coords['max_lon'], maxy = clip_coords['max_lat']) 
                                                          for i in range(len(o_bands_data)) ]

        # Get the timestamp from the yaml file.
        timestamp = datetime.strptime(yml_meta['extent']['center_dt'], '%Y-%m-%d %H:%M:%S')

        # Stack the bands together into a single xarray dataset.
        band_data = stack_bands(o_bands_data, measurement, timestamp)

        # Append each stacked scene to a list to be combined later.
        array_list.append(band_data)

    # Stack the scenes together into xarray dataset.
    ds = stack_scenes(array_list)

    # Mask out nodata values.
    ds = ds.where(ds != -9999)
    print(f'Final Dataset: {ds}')

    return ds

## Set Up Parameters

In [5]:
# St Maarten bounding box to subset the data
clip_coords = {'min_lon':-63.461424,
               'min_lat': 17.950000,
               'max_lon': -62.80000,
               'max_lat': 18.334848}

# Set size of dask chunks to use for the scenes
dask_chunks = dict(
    x = 1000,
    y = 1000
)

# Set mosaic type (options: 'median', 'mean', 'max', 'min')
mosaic_type = 'median'


baseline_measurement = ["blue", "green", "red", "nir", "swir1", "swir2", "pixel_qa"]
baseline_product = 'landsat_8'

analysis_measurement = ["blue", "green", "red", "nir", "swir1", "swir2", "pixel_qa"]
analysis_product = 'landsat_8'


#parameter display_name="Water Mask" description="If you would like the water to be masked out choose YES, if you would like the full image choose NO" datatype="string" options=["YES", "NO"],
mask_water = 'YES'


## Load Data

In [6]:
def prep_dataset(in_dir, measurement, product, clip_coords = None):
    """Prepare either the baseline or analysis dataset."""
    scenes = glob.glob(f'{in_dir}/*/')

    array_list = []

    for scene in scenes:
        yml = f'{scene}/datacube-metadata.yaml'
        with open (yml) as stream: yml_meta = yaml.safe_load(stream)

        # Load the bands provided in 'measurement' from the yaml file.
        o_bands_data = [ rxr.open_rasterio(scene + yml_meta['image']['bands'][b]['path'], chunks=dask_chunks) for b in measurement ] 

        # Clip the data to the bounding box if provided.
        if clip_coords is not None:
            o_bands_data = [ o_bands_data[i].rio.clip_box(minx = clip_coords['min_lon'], miny = clip_coords['min_lat'], 
                                                          maxx = clip_coords['max_lon'], maxy = clip_coords['max_lat']) 
                                                          for i in range(len(o_bands_data)) ]

        # Get the timestamp from the yaml file.
        timestamp = datetime.strptime(yml_meta['extent']['center_dt'], '%Y-%m-%d %H:%M:%S')

        # Stack the bands together into a single xarray dataset.
        band_data = stack_bands(o_bands_data, measurement, timestamp)

        # Append each stacked scene to a list to be combined later.
        array_list.append(band_data)

    # Stack the scenes together into xarray dataset.
    ds = stack_scenes(array_list)

    # Mask out nodata values.
    ds = ds.where(ds != -9999)
    print(f'Final Dataset: {ds}')

    return ds

In [7]:
# Running on data from St Maarten
baseline_dir = '/home/spatialdays/Documents/ARD_Data/StMaarten_Landsat_baseline/'
baseline_ds = prep_dataset(baseline_dir, baseline_measurement, baseline_product, clip_coords)

Final Dataset: <xarray.Dataset>
Dimensions:      (time: 4, y: 1429, x: 2455)
Coordinates:
    spatial_ref  int64 0
  * x            (x) float64 -63.46 -63.46 -63.46 -63.46 ... -62.8 -62.8 -62.8
  * y            (y) float64 18.33 18.33 18.33 18.33 ... 17.95 17.95 17.95 17.95
  * time         (time) datetime64[ns] 2018-09-06 2018-11-09 ... 2018-03-14
Data variables:
    blue         (time, y, x) float32 dask.array<chunksize=(1, 1000, 1000), meta=np.ndarray>
    green        (time, y, x) float32 dask.array<chunksize=(1, 1000, 1000), meta=np.ndarray>
    red          (time, y, x) float32 dask.array<chunksize=(1, 1000, 1000), meta=np.ndarray>
    nir          (time, y, x) float32 dask.array<chunksize=(1, 1000, 1000), meta=np.ndarray>
    swir1        (time, y, x) float32 dask.array<chunksize=(1, 1000, 1000), meta=np.ndarray>
    swir2        (time, y, x) float32 dask.array<chunksize=(1, 1000, 1000), meta=np.ndarray>
    pixel_qa     (time, y, x) float64 dask.array<chunksize=(1, 1000, 1000),

In [8]:
# St Maarten Analysis
analysis_dir = '/home/spatialdays/Documents/ARD_Data/StMaarten_Landsat_analysis/'
analysis_ds = prep_dataset(analysis_dir, analysis_measurement, analysis_product, clip_coords)

Final Dataset: <xarray.Dataset>
Dimensions:      (time: 3, y: 1429, x: 2455)
Coordinates:
    spatial_ref  int64 0
  * x            (x) float64 -63.46 -63.46 -63.46 -63.46 ... -62.8 -62.8 -62.8
  * y            (y) float64 18.33 18.33 18.33 18.33 ... 17.95 17.95 17.95 17.95
  * time         (time) datetime64[ns] 2022-11-20 2022-12-22 2022-10-19
Data variables:
    blue         (time, y, x) float32 dask.array<chunksize=(1, 1000, 1000), meta=np.ndarray>
    green        (time, y, x) float32 dask.array<chunksize=(1, 1000, 1000), meta=np.ndarray>
    red          (time, y, x) float32 dask.array<chunksize=(1, 1000, 1000), meta=np.ndarray>
    nir          (time, y, x) float32 dask.array<chunksize=(1, 1000, 1000), meta=np.ndarray>
    swir1        (time, y, x) float32 dask.array<chunksize=(1, 1000, 1000), meta=np.ndarray>
    swir2        (time, y, x) float32 dask.array<chunksize=(1, 1000, 1000), meta=np.ndarray>
    pixel_qa     (time, y, x) float64 dask.array<chunksize=(1, 1000, 1000), met

## Cloud Masking

In [9]:
# Generate clean mask
clean_mask_baseline = ls_clean_mask(baseline_ds, keep_water=False)
clean_mask_analysis = ls_clean_mask(analysis_ds, keep_water=False)

## Generate Geomedians

In [10]:
from odc.algo import xr_geomedian, to_rgba, to_f32
from odc.ui import to_png_data
#import hdstats

# Drop the pixel_qa bands from the datasets
ds_baseline = baseline_ds.drop(['pixel_qa'])
ds_analysis = analysis_ds.drop(['pixel_qa'])

scale, offset = (1, 0)

# Apply the clean mask to the baseline dataset and create geomedian
ds_baseline_clean = ds_baseline.where(clean_mask_baseline == 1, np.nan)
ds_clean_32_baseline = to_f32(ds_baseline_clean, scale=scale, offset=offset)
geomedian_baseline = xr_geomedian(ds_clean_32_baseline, 
                  num_threads=1,  # disable internal threading, dask will run several concurrently
                  #axis='time',
                  eps=0.2*(1/10_000),  # 1/5 pixel value resolution
                  nocheck=True) 

# Apply the clean mask to the analysis dataset and create geomedian
ds_analysis_clean = ds_analysis.where(clean_mask_analysis == 1, np.nan)
ds_clean_32_analysis = to_f32(ds_analysis_clean, scale=scale, offset=offset)
geomedian_analysis = xr_geomedian(ds_clean_32_analysis, 
                  num_threads=1,  # disable internal threading, dask will run several concurrently
                  #axis='time',
                  eps=0.2*(1/10_000),  # 1/5 pixel value resolution
                  nocheck=True)



## Fractional Cover from Geomedian

In [11]:
# Run fractional cover classifier on the geomedian dataset (baseline)
geomedian_baseline = geomedian_baseline.rename({'x':'longitude', 'y':'latitude'})
frac_classes_baseline = frac_coverage_classify(geomedian_baseline)

/home/spatialdays/Documents/product-notebooks/AncillaryData/endmembers_landsat.csv


/home/spatialdays/anaconda3/envs/notebooks/lib/python3.11/site-packages/odc/algo/_geomedian.py:99: RuntimeWarning: Mean of empty slice
  lambda x: nangeomedian_pcm(x, **kw),
/home/spatialdays/anaconda3/envs/notebooks/lib/python3.11/site-packages/odc/algo/_geomedian.py:99: RuntimeWarning: Mean of empty slice
  lambda x: nangeomedian_pcm(x, **kw),
/home/spatialdays/anaconda3/envs/notebooks/lib/python3.11/site-packages/odc/algo/_geomedian.py:99: RuntimeWarning: Mean of empty slice
  lambda x: nangeomedian_pcm(x, **kw),
/home/spatialdays/anaconda3/envs/notebooks/lib/python3.11/site-packages/odc/algo/_geomedian.py:99: RuntimeWarning: Mean of empty slice
  lambda x: nangeomedian_pcm(x, **kw),
/home/spatialdays/anaconda3/envs/notebooks/lib/python3.11/site-packages/odc/algo/_geomedian.py:99: RuntimeWarning: Mean of empty slice
  lambda x: nangeomedian_pcm(x, **kw),
/home/spatialdays/anaconda3/envs/notebooks/lib/python3.11/site-packages/odc/algo/_geomedian.py:99: RuntimeWarning: Mean of empty s

In [12]:
# Run fractional cover classifier on the geomedian dataset (analysis)
geomedian_analysis = geomedian_analysis.rename({'x':'longitude', 'y':'latitude'})
frac_classes_analysis = frac_coverage_classify(geomedian_analysis)

/home/spatialdays/Documents/product-notebooks/AncillaryData/endmembers_landsat.csv


/home/spatialdays/anaconda3/envs/notebooks/lib/python3.11/site-packages/odc/algo/_geomedian.py:99: RuntimeWarning: Mean of empty slice
  lambda x: nangeomedian_pcm(x, **kw),
/home/spatialdays/anaconda3/envs/notebooks/lib/python3.11/site-packages/odc/algo/_geomedian.py:99: RuntimeWarning: Mean of empty slice
  lambda x: nangeomedian_pcm(x, **kw),
/home/spatialdays/anaconda3/envs/notebooks/lib/python3.11/site-packages/odc/algo/_geomedian.py:99: RuntimeWarning: Mean of empty slice
  lambda x: nangeomedian_pcm(x, **kw),
/home/spatialdays/anaconda3/envs/notebooks/lib/python3.11/site-packages/odc/algo/_geomedian.py:99: RuntimeWarning: Mean of empty slice
  lambda x: nangeomedian_pcm(x, **kw),
/home/spatialdays/anaconda3/envs/notebooks/lib/python3.11/site-packages/odc/algo/_geomedian.py:99: RuntimeWarning: Mean of empty slice
  lambda x: nangeomedian_pcm(x, **kw),
/home/spatialdays/anaconda3/envs/notebooks/lib/python3.11/site-packages/odc/algo/_geomedian.py:99: RuntimeWarning: Mean of empty s